In [2]:
"""Redirecting writing

If using a library that can print messages to the console, editing the library
by  replacing `print()` with `tqdm.write()` may not be desirable.
In that case, redirecting `sys.stdout` to `tqdm.write()` is an option.

To redirect `sys.stdout`, create a file-like class that will write
any input string to `tqdm.write()`, and supply the arguments
`file=sys.stdout, dynamic_ncols=True`.

A reusable canonical example is given below:
"""
from __future__ import print_function
from time import sleep
import contextlib
import sys
from tqdm import tqdm


class DummyTqdmFile(object):
    """Dummy file-like that will write to tqdm"""
    file = None

    def __init__(self, file):
        self.file = file

    def write(self, x):
        # Avoid print() second call (useless \n)
        if len(x.rstrip()) > 0:
            tqdm.write(x, file=self.file)

    def flush(self):
        return getattr(self.file, "flush", lambda: None)()


@contextlib.contextmanager
def std_out_err_redirect_tqdm():
    orig_out_err = sys.stdout, sys.stderr
    try:
        # sys.stdout = sys.stderr = DummyTqdmFile(orig_out_err[0])
        sys.stdout, sys.stderr = map(DummyTqdmFile, orig_out_err)
        yield orig_out_err[0]
    # Relay exceptions
    except Exception as exc:
        raise exc
    # Always restore sys.stdout/err if necessary
    finally:
        sys.stdout, sys.stderr = orig_out_err


def some_fun(i):
    print("Fee, fi, fo,".split()[i])


# Redirect stdout to tqdm.write()
with std_out_err_redirect_tqdm() as orig_stdout:
    # tqdm needs the original stdout
    # and dynamic_ncols=True to autodetect console width
    for i in tqdm(range(3), file=orig_stdout, dynamic_ncols=True):
        # order of the following two lines should not matter
        some_fun(i)
        sleep(.5)

# After the `with`, printing is restored
print("Done!")

Fee,
fi,
fo,
100%|██████████| 3/3 [00:01<00:00,  1.96it/s]
Done!


In [3]:
"""
# Simple tqdm examples and profiling

# Benchmark
for i in _range(int(1e8)):
    pass

# Basic demo
import tqdm
for i in tqdm.trange(int(1e8)):
    pass

# Some decorations
import tqdm
for i in tqdm.trange(int(1e8), miniters=int(1e6), ascii=True,
                     desc="cool", dynamic_ncols=True):
    pass

# Nested bars
from tqdm import trange
for i in trange(10):
    for j in trange(int(1e7), leave=False, unit_scale=True):
        pass

# Experimental GUI demo
import tqdm
for i in tqdm.tgrange(int(1e8)):
    pass

# Comparison to https://code.google.com/p/python-progressbar/
try:
    from progressbar.progressbar import ProgressBar
except ImportError:
    pass
else:
    for i in ProgressBar()(_range(int(1e8))):
        pass

# Dynamic miniters benchmark
from tqdm import trange
for i in trange(int(1e8), miniters=None, mininterval=0.1, smoothing=0):
    pass

# Fixed miniters benchmark
from tqdm import trange
for i in trange(int(1e8), miniters=4500000, mininterval=0.1, smoothing=0):
    pass
"""

from time import sleep
from timeit import timeit
import re

# Simple demo
from tqdm import trange
for i in trange(16, leave=True):
    sleep(0.1)

# Profiling/overhead tests
stmts = filter(None, re.split(r'\n\s*#.*?\n', __doc__))
for s in stmts:
    print(s.replace('import tqdm\n', ''))
    print(timeit(stmt='try:\n\t_range = xrange'
                 '\nexcept:\n\t_range = range\n' + s, number=1), 'seconds')

100%|██████████| 16/16 [00:01<00:00,  9.63it/s]


for i in _range(int(1e8)):
    pass


  1%|          | 658383/100000000 [00:00<00:30, 3291553.22it/s]

2.385589327896014 seconds
for i in tqdm.trange(int(1e8)):
    pass


cool:   1%|1         | 1000000/100000000 [00:00<00:17, 5526449.01it/s]

...


  6%|▋         | 631k/10.0M [00:00<00:02, 3.14Mit/s]
 10%|▉         | 984k/10.0M [00:00<00:02, 3.27Mit/s]
 14%|█▎        | 1.36M/10.0M [00:00<00:02, 3.39Mit/s]
 17%|█▋        | 1.74M/10.0M [00:00<00:02, 3.48Mit/s]
 21%|██▏       | 2.15M/10.0M [00:00<00:02, 3.57Mit/s]
 26%|██▌       | 2.57M/10.0M [00:00<00:02, 3.67Mit/s]
 30%|███       | 3.01M/10.0M [00:00<00:01, 3.75Mit/s]
 34%|███▍      | 3.43M/10.0M [00:00<00:01, 3.81Mit/s]
 39%|███▉      | 3.88M/10.0M [00:01<00:01, 3.87Mit/s]
 43%|████▎     | 4.33M/10.0M [00:01<00:01, 3.93Mit/s]
 48%|████▊     | 4.79M/10.0M [00:01<00:01, 3.99Mit/s]
 53%|█████▎    | 5.25M/10.0M [00:01<00:01, 4.04Mit/s]
 57%|█████▋    | 5.71M/10.0M [00:01<00:01, 4.08Mit/s]
 62%|██████▏   | 6.17M/10.0M [00:01<00:00, 4.11Mit/s]
 66%|██████▋   | 6.64M/10.0M [00:01<00:00, 4.15Mit/s]
 71%|███████   | 7.11M/10.0M [00:01<00:00, 4.18Mit/s]
 76%|███████▌  | 7.58M/10.0M [00:01<00:00, 4.21Mit/s]
 80%|████████  | 8.05M/10.0M [00:01<00:00, 4.23Mit/s]
 85%|████████▌ | 8.52M/10.0M 

23.64854876010213 seconds
for i in tqdm.tgrange(int(1e8)):
    pass


/Users/wcz/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/tqdm/_tqdm_gui.py:45: TqdmExperimentalWarning: GUI is experimental/alpha
  warn('GUI is experimental/alpha', TqdmExperimentalWarning)


<Figure size 900x220 with 1 Axes>

KeyboardInterrupt: 

In [12]:
import tqdm
for i in tqdm.trange(int(1e8), miniters=int(1e6), ascii=True,
                     desc="cool", dynamic_ncols=True):
    pass



cool:  62%|######2   | 62000000/100000000 [00:11<00:07, 5170388.96it/s]

KeyboardInterrupt: 

In [9]:
from time import sleep
# from tqdm import tqdm
import tqdm

pbar = tqdm.tqdm(["a", "b", "c", "d"])
# pbar = tqdm.tqdm_gui(["a", "b", "c", "d"])
for char in pbar:
#     pbar.set_description("Processing %s" % char)
    sleep(0.5)
    i = ord(char)
    pbar.set_postfix({'i': i})


100%|██████████| 4/4 [00:02<00:00,  1.97it/s, i=100]
